In [24]:
#Import all necessary packages.
%matplotlib inline
import sys, platform, os

from matplotlib import pyplot as plt
import numpy as np

import camb
from camb import model, initialpower

import pylab as pl

import scipy
from scipy.interpolate import interp1d
from __future__ import division

from scipy import integrate
from scipy import linalg

In [25]:
font = {'size'   : 16, 'family':'STIXGeneral'}
axislabelfontsize='x-large'
plt.rc('font', **font)
plt.rcParams['legend.fontsize']='medium'

$$ H_0 = 0.678 $$ $$ \Omega_0 = \Omega_B + \Omega_C $$ $$ h = \frac{H_0}{100} $$

In [26]:
#Fiducial cosmological parameters
c=3e5
pi=np.pi

hubble=0.678
omegab=0.022*pow(hubble,-2)
omegac=0.119*pow(hubble,-2)
om0=omegac+omegab
H00=100*hubble
Ass=2.14e-9
nss = 0.968

gamma=0.545

In [27]:
#Set up the fiducial cosmology
pars = camb.CAMBparams()
#Set cosmology
pars.set_cosmology(H0=H00, ombh2=omegab*pow(hubble,2), omch2=omegac*pow(hubble,2),omk=0,mnu=0)
pars.set_dark_energy() #LCDM (default)
pars.InitPower.set_params(ns=nss, r=0, As=Ass)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [28]:
#calculate results for these parameters
results = camb.get_results(pars)

In [29]:
#Get matter power spectrum at z=0: P(k,z=0)
pars.set_matter_power(redshifts=[0.], kmax=2.0)

#Linear spectra
pars.NonLinear = model.NonLinear_none
results.calc_power_spectra(pars)
kh, z, pk = results.get_matter_power_spectrum(minkh=1e-4, maxkh=2.0, npoints = 200)

In [30]:
#Construct P(k,z=0) interpolating function, in units of Mpc (no h)
Pkz0 = interp1d(kh*hubble, pk[0]/pow(hubble,3))

In [31]:
#Redshift bins
zlist = np.arange(0.1,1.45,0.1)
ztest = zlist[4]
Nzbins = len(zlist)

print (zlist)
print ("ztest =", ztest)
print ("Number of redshift bins =", Nzbins)

[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4]
ztest = 0.5
Number of redshift bins = 14


$$ E(z) = \frac{H(z)}{H_0} = \sqrt{1-\Omega_0+\Omega_0(1+z)^3} $$

In [32]:
#Define E(z) = H(z)/H0
def Ez(zc):
    return np.sqrt(1-om0+om0*pow(1+zc,3))

$$ r(z) = \int_{0}^{z} \frac{c}{hE(z)} $$

In [33]:
## Define the comoving distance
def drdz(zp):
    return (c/H00)/Ez(zp)
def rcom(zc):
    return scipy.integrate.romberg(drdz,0,zc)

print (rcom(ztest))

1946.7257776169436


Factor de crecimiento lineal
$$ f = \Bigg(\frac{\Omega_0(1+z)^3}{1-\Omega_0 + \Omega_0(1+z)^3}\Bigg)^\gamma $$

In [34]:
#Define the growth function in LCDM
def fg(zz):
    omz=om0*pow(1+zz,3)/(om0*pow(1+zz,3)+1-om0)
    return pow(omz,gamma)

print (fg(ztest))

0.756249102937079


Factor de crecimiento total
$$ D(z) = \int_{0}^{z} \frac{f(z^´)}{(1+z^´)} dz^´ $$

In [35]:
#Get the growth factor 
def Dg_dz(zz):
    return fg(zz)/(1+zz)
def Dgz(zc):
    ans = scipy.integrate.romberg(Dg_dz, 0.0, zc)
    return np.exp(-ans)

print (Dgz(ztest))

0.7701490452523118


$$\Omega_{HI}$$ $$b_{HI}$$

In [36]:
def OmHI(zc):
    return 0.00048+0.00039*zc-0.000065*pow(zc,2)

def bHI(zc):
    return 0.67+0.18*zc+0.05*pow(zc,2)

$$P_m(k,z)$$

In [37]:
def Pkz(kk,zc):    
    return pow(Dgz(zc),2)*Pkz0(kk)

$$T(z)$$

In [38]:
def Tb(zc): #in mK
    return 0.0559+0.2324*zc-0.024*pow(zc,2)

print (ztest, Tb(ztest))

0.5 0.1661


$$P(k,z) = T(z)^2 b_{HI}^2 P_m(k,z)$$

In [39]:
#Construct P_HI(k,z) [mK^2]
def PHI(kk,zc):
    return pow(Tb(zc),2)*pow(bHI(zc),2)*Pkz(kk,zc)

$$\theta_B = \frac{21(1+z)}{D_{dish}}$$ $$\Omega_{pix} = 1.13 \theta_B^2$$

In [40]:
Ndishes=64
Ddish=13.5*100 #cm
Nbeams=1

def thetab(zc):
    return 21*(1+zc)/Ddish

def omegapix(zc):
    return 1.13*pow(thetab(zc),2)

$$\sigma_{pix} = \frac{T_{sys}}{\sqrt{\Delta f*t_{total}\frac{\Omega_{pix}}{\Omega_{tot}}N_{dishes}N_{beams}}}$$

In [41]:
Area=4000.0 #deg^2
omegatot = Area*pow(pi/180,2)
ttotal = 4000*60*60*(Area/4000) #4000 hrs for 4000 deg^2

def fc(zc):  #frecuencia observada
    return 1420.4/(1+zc)

def Tsky(zc):
    return 60*pow(300/fc(zc),2.55)*1e3

#receiver temperature from specs document
Tsyslist = [23.5*1e3,23.0*1e3,23.0*1e3,28.0*1e3,29.0*1e3,30.0*1e3,28.5*1e3,29.5*1e3,\
            31.0*1e3,33.0*1e3,34.0*1e3,35.0*1e3,37.0*1e3,38.0*1e3] #mK

def tobs(zc):
    return ttotal*(omegapix(zc)/omegatot)*Ndishes*Nbeams

Dzbin = 0.1
dfpix = 50*1e3 #Hz
midfreq = 1420.4e6 #Hz #frecuencia emitida

def dzpix(zc):
    return pow(1+zc,2)*dfpix/midfreq
def sigpix(zc,Tsys):
    return Tsys/np.sqrt(dfpix*tobs(zc)) 

$$V_{pix} = \Omega_{pix}\int_{0}^{z} \frac{c[r(z^´)]^2}{hE(z^´)} dz^´ $$

In [42]:
def dVpixdz(zz):    
    return c*pow(rcom(zz),2)/(H00*Ez(zz))
def Vpix(zc):
    return omegapix(zc)*scipy.integrate.romberg(dVpixdz,zc-dzpix(zc)/2,zc+dzpix(zc)/2)

$$ W(k)^2 = exp \Bigg[ -k^2r(z)^2 \Bigg( \frac{\Omega_B}{\sqrt{8ln(2)}} \Bigg)^2 \Bigg] $$

In [43]:
def Wsq(kk,zc): #very rough modelling, it should be Wsq(mu,kk,zc) - exercise!
    return np.exp(-pow(kk,2)*pow(rcom(zc),2)*pow(thetab(zc),2)/(8*np.log(2)))

$$P^N(k) = \sigma_{pix}^2V_{pix}^2W^-2(k)$$

In [44]:
def Pnoise(kk,zc,Tsys):
    return pow(sigpix(zc,Tsys),2)*Vpix(zc)*pow(Wsq(kk,zc),-1.)

$$ k_{min} = \frac{2\pi}{\sqrt{r(z)^2\Omega_{tot}}} $$  $$ k_{max} = 0,14(1+z)^{\frac{2}{2+ns}} $$

In [45]:
def kmin(zc):
    return 2*pi/np.sqrt(pow(rcom(zc),2)*omegatot)
def kmax(zc):
    return 0.14*pow(1+zc,2/(2+nss)); #non-linear cutoff Smith et al 2003

print (round(kmin(ztest),3), round(kmax(ztest),2))

0.003 0.18


$$V_{sur} = \Omega_{sur}\int_{0}^{z} \frac{c[r(z^´)]^2}{hE(z^´)} dz^´ $$

In [46]:
#survey (bin) volume [Mpc^3]
def dVsurdz(zz):    
    return omegatot*c*pow(rcom(zz),2)/(H00*Ez(zz))
    
def Vsur(zc):
    return scipy.integrate.romberg(dVsurdz,zc-Dzbin/2,zc+Dzbin/2)

$$ V_{eff} = V_{sur} \Bigg( \frac{P_{HI}}{(P_{HI}+P^N)} \Bigg)^2 $$

In [47]:
#effective volume going in the Fisher matrix
def Veff(kk,zc):
    return Vsur(zc)*(PHI(kk,zc)/(PHI(kk,zc)+Pnoise(kk,zc,Tsys)))**2

print ("%.4g" % Vsur(ztest))

1.554e+09


In [48]:
def dlnP_dlnOmHIbHI(kk,zc):
    return 2.0

In [49]:
def dFdk(kk):
    return (1./(4*pi*pi))*pow(kk,2)*pow(dlnP_dlnOmHIbHI(kk,zc),2)*Veff(kk,zc)

$$ F_{ij} = \frac{1}{4\pi^2} \int_{k_{min}}^{k_{max}} k^2dk \big[ \delta_ilnP_{HI} \delta_jlnP_{HI} \big] V_{eff} $$

In [50]:
for i in range(0,Nzbins):
    zc = round(zlist[i],2)
    Tsys = Tsyslist[i]+Tsky(zc)
    K = np.linspace(kmin(zc), kmax(zc), 100)
    dF = dFdk(K)
    Fisher = scipy.integrate.simps(dF,K) 
    #since we only vary one parameter, we don't need the covariance matrix
    print (zc, round(np.sqrt(1/Fisher),3))

0.1 0.01
0.2 0.005
0.3 0.005
0.4 0.007
0.5 0.009
0.6 0.011
0.7 0.013
0.8 0.015
0.9 0.018
1.0 0.022
1.1 0.026
1.2 0.03
1.3 0.036
1.4 0.041
